# Machine Learning Project Checklist

1. Frame the problem and look at the big picture.

2. **Get the data.**

3. Explore the data to gain insights.

4. Explore many different models and short-list the best ones.

5. Fine-tune your models and combine them into a great solution.

6. Present your solution.

In [1]:
import os

In [2]:
REPO_DIR = os.path.join(os.environ['USERPROFILE'], 'repos')
PROJ_DIR = os.path.join(REPO_DIR, 'real_estate_machine_learning')
os.chdir(PROJ_DIR)

In [3]:
DATA_DIR = './data'
EXT_DIR = os.path.join(DATA_DIR, 'ext')
INT_DIR = os.path.join(DATA_DIR, 'interim')
IMG_DIR = './img'
INPUT_DIR = '../real_estate_hungary/output'
FILENAME = 'ForSaleRent_20181101.csv'
SCRAPE_DATE = os.path.splitext(FILENAME)[0].split('_')[1]
INPUT_FILEPATH=os.path.join(INPUT_DIR, FILENAME)

In [4]:
import pandas as pd, numpy as np
from datetime import datetime
import src.processing as proc
from src.processing import extract_num
from src.utils import *

## Scraped data
I have written a Python script, based on my module [real_estate_hungary](https://github.com/tszereny/real_estate_hungary, "tszereny's GitHub page"), which extracts pieces of information from one of the most popular Hungarian [real estate website](https://ingatlan.com/, "https://ingatlan.com"). In short it turns the data on the website into tabular form.  
The scraped dataset contains more than 50,000 records of real estate properties in Budapest, the capital city of Hungary.

In [5]:
print('Data has been scraped in {}.'.format(datetime.strptime(SCRAPE_DATE, '%Y%m%d').strftime('%B, %Y')))

Data has been scraped in November, 2018.


In [6]:
na_hun_equivalent='nincs megadva'

In [7]:
raw=pd.read_csv(INPUT_FILEPATH, encoding='utf8', na_values=na_hun_equivalent)

Translate column names from Hungarian to English

In [8]:
raw=proc.transform_naming(raw)

### Avilable columns
Three categories:
1. Unique ID
2. Possible features
3. Technical columns

In [9]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55146 entries, 0 to 55145
Data columns (total 60 columns):
address                     55120 non-null object
accessibility               14456 non-null object
batch_num                   55146 non-null int64
ceiling_height              37722 non-null object
buses                       36823 non-null object
buses_count                 36823 non-null float64
furnished                   8593 non-null object
cluster_id                  55146 non-null object
property_id                 55146 non-null int64
desc                        54531 non-null object
city_district               55146 non-null int64
smoking                     4822 non-null object
floors                      50258 non-null object
energy_perf_cert            2809 non-null object
balcony                     23573 non-null object
bath_and_wc                 29760 non-null object
type_of_heating             49418 non-null object
equipped                    7926 non-null obje

In [10]:
composite_id=['property_id', 'timestamp']

In [11]:
technical=['property_url', 'cluster_id', 'batch_num', 'page_num', 'max_page', 'max_listing', 'is_ad_active']

In [12]:
possible_features=raw.columns[~raw.columns.isin(technical)&~raw.columns.isin(composite_id)].tolist()

Check whether the scraped dataset is distinct

In [13]:
unique_records=raw[[composite_id[0]]+possible_features].drop_duplicates()
unique_records_idx=unique_records.index
total_records=len(raw)
duplicated_records_ratio=1 - len(unique_records)/total_records
print('Total number of records: {0:,}'.format(total_records))
print('Duplicated records are {0:.3%}'.format(duplicated_records_ratio))

Total number of records: 55,146
Duplicated records are 2.675%


Filter for unique records only and exclude technical columns

In [14]:
u_raw=raw.loc[unique_records_idx, possible_features + composite_id]

### Elevation (metre)
Merging elevation column to scraped dataset

In [15]:
elevation = pd.read_csv(os.path.join(EXT_DIR, 'elevation.csv'), encoding='utf8')

In [16]:
elevation.head()

,elevation,latitude,longitude
0,102,47.460680,19.048690
1,106,47.529858,19.079060
2,106,47.529730,19.078869
3,102,47.538403,19.064398
4,102,47.546740,19.066140


In [17]:
u_raw[['lat', 'lng']].drop_duplicates().head()

,lat,lng
0,47.460680,19.048690
1,47.529858,19.079060
2,47.529730,19.078869
7,47.538403,19.064398
10,47.546740,19.066140


In [18]:
u_raw = u_raw.merge(how='left',
                    right=elevation,
                    left_on=['lat', 'lng'],
                    right_on=['latitude', 'longitude'])
u_raw.drop(['latitude', 'longitude'], axis=1, inplace=True)

Check whether any null value in the dataset:

In [19]:
print('Is there any null value? {}'.format(np.any(u_raw[['lat', 'lng', 'elevation']].isnull())))

Is there any null value? False


In [20]:
possible_features += ['elevation']

In [21]:
u_raw.head()

,address,accessibility,ceiling_height,buses,buses_count,furnished,desc,city_district,smoking,floors,...,orientation,trams,trams_count,all_night_services,all_night_services_count,year_built,building_floors,property_id,timestamp,elevation
0,Budai Bolero II,igen,NaN,103|133E|33,3.0,NaN,| Exkluzív otthon az Ön igényeire szabva! A Bu...,11,NaN,10,...,nyugat,1,1.0,NaN,NaN,2019,10,26313868,2018-11-03 08:23:13.955910,102
1,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,földszint,...,északnyugat,1|14,2.0,914|914A,2.0,NaN,5,24714938,2018-11-03 08:23:16.124314,106
2,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,2,...,délnyugat,1|14,2.0,914|914A,2.0,2018,5,25561892,2018-11-03 08:23:18.277117,106
3,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,1,...,északnyugat,1|14,2.0,914|914A,2.0,2018,5,27741740,2018-11-03 08:23:20.133521,106
4,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,5,...,északkelet,1|14,2.0,914|914A,2.0,2018,5,26996343,2018-11-03 08:23:22.161524,106


### Unique ID
- Ading unique id
- Composed from **property_id** and **timestamp**

In [22]:
ts_fmt='%Y-%m-%d %H:%M:%S.%f'
u_raw['id']=u_raw['property_id']+u_raw['timestamp'].apply(lambda ts: datetime.strptime(ts, ts_fmt).timestamp())

In [23]:
u_raw.loc[:,possible_features].head()

,address,accessibility,ceiling_height,buses,buses_count,furnished,desc,city_district,smoking,floors,...,trolley_buses_count,listing_type,orientation,trams,trams_count,all_night_services,all_night_services_count,year_built,building_floors,elevation
0,Budai Bolero II,igen,NaN,103|133E|33,3.0,NaN,| Exkluzív otthon az Ön igényeire szabva! A Bu...,11,NaN,10,...,NaN,for-sale,nyugat,1,1.0,NaN,NaN,2019,10,102
1,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,földszint,...,NaN,for-sale,északnyugat,1|14,2.0,914|914A,2.0,NaN,5,106
2,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,2,...,NaN,for-sale,délnyugat,1|14,2.0,914|914A,2.0,2018,5,106
3,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,1,...,NaN,for-sale,északnyugat,1|14,2.0,914|914A,2.0,2018,5,106
4,Csata utca 30.,NaN,3 m-nél alacsonyabb,14|32|105,3.0,NaN,| XIII kerület közkedvelt részén az Árpád-hí...,13,NaN,5,...,NaN,for-sale,északkelet,1|14,2.0,914|914A,2.0,2018,5,106


### Available listing types
- for-sale
- for-rent

In [24]:
sale_flt=u_raw.listing_type=='for-sale'
rent_flt=u_raw.listing_type=='for-rent'

### Measurement scales
Data types of the features/columns should be modified accordingly:

|Measurement scale|Representation|Data type|
|-----------------|--------------|---------|
|Nominal, category|text|str|
|Ordinal|text|str|
|Interval|numeric|int, float|
|Ratio|numeric|int, float|

I have picked all the interval/ratio and nominal scaled columns based on their values

In [25]:
interval_or_ratio = ['lat', 'lng',
                     'elevation', 'price_in_huf',
                     'area_size', 'room', 
                     'balcony', 'parking_lot_price',
                     'utilities', 'min_tenancy',
                     'metro_lines_count', 'trams_count',
                     'trolley_buses_count', 'buses_count',
                     'boats_count', 'local_railways_count',
                     'all_night_services_count']

In [26]:
nominal = u_raw[possible_features].columns[~u_raw[possible_features].columns.isin(interval_or_ratio)].tolist()
nominal

['address',
 'accessibility',
 'ceiling_height',
 'buses',
 'furnished',
 'desc',
 'city_district',
 'smoking',
 'floors',
 'energy_perf_cert',
 'bath_and_wc',
 'type_of_heating',
 'equipped',
 'boats',
 'local_railways',
 'condition_of_real_estate',
 'with_entry_to_garden',
 'view',
 'pets',
 'convenience_level',
 'vacant',
 'residental_park_name',
 'lift',
 'air_conditioned',
 'metro_lines',
 'panelprogram',
 'parking',
 'attic',
 'trolley_buses',
 'listing_type',
 'orientation',
 'trams',
 'all_night_services',
 'year_built',
 'building_floors']

As a next step, data type of the columns should be checked and if it is not right, fix it:
- Cast it directly, if it is possible
- Extract the value from the text, then cast it

In [27]:
nominal_not_str = {k: 'object' for k, v in u_raw[nominal].dtypes.items() if v != 'object'}
print('{} column(s) have incorrect data type!'.format(', '.join(nominal_not_str.keys())))

city_district column(s) have incorrect data type!


In [28]:
u_raw = u_raw.astype(nominal_not_str)

In [29]:
interval_or_ratio_str = [k for k, v in u_raw[interval_or_ratio].dtypes.items() if v == 'object']
print('{} column(s) have incorrect data type!'.format(', '.join(interval_or_ratio_str)))

price_in_huf, area_size, room, balcony, parking_lot_price, utilities, min_tenancy column(s) have incorrect data type!


In [30]:
u_raw.loc[sale_flt, interval_or_ratio_str].head()

,price_in_huf,area_size,room,balcony,parking_lot_price,utilities,min_tenancy
0,69.9 millió Ft,92 m²,3 + 1 fél,18.24 m²,3.2 M Ft,NaN,NaN
1,50.3 millió Ft,75 m²,3,6.52 m²,2.89 M Ft,NaN,NaN
2,49.5 millió Ft,75 m²,3,7 m²,2.89 M Ft,NaN,NaN
3,46.1 millió Ft,71 m²,3,10.29 m²,2.9 M Ft,NaN,NaN
4,62.9 millió Ft,77 m²,3,30 m²,2.89 M Ft,NaN,NaN


In [31]:
u_raw.loc[rent_flt, interval_or_ratio_str].head()

,price_in_huf,area_size,room,balcony,parking_lot_price,utilities,min_tenancy
20,140 000 Ft,47 m²,1 + 1 fél,NaN,NaN,9 900 Ft/hó,1 év
21,110 000 Ft,30 m²,1,NaN,NaN,20 000 Ft/hó,1 év
22,160 000 Ft,42 m²,2,NaN,NaN,NaN,1 év
23,160 000 Ft,49 m²,1 + 1 fél,NaN,NaN,25 000 Ft/hó,1 év
24,450 000 Ft,90 m²,1 + 1 fél,3 m²,NaN,80 000 Ft/hó,1 év


Columns above, should be converted to numeric, according to different listing type (for-sale, for-rent) it may have different value embedded in the text. e.g.:

### Convert strings to numeric
- Price - three categories:
    1. Billion (milliárd)
    2. Million (millió)
    3. HUF, Hungarian Forint (Ft)<br><br>
- Area size:
    - Remove m$^2$ (square meter)<br><br>
- Number of rooms: in the current representation they can't be analyzed, therefore splitting into two category
    1. Number of full rooms (room >12 m$^2$)
    2. Number of half rooms (room <12 m$^2$)
    3. Adding total number of rooms<br><br>
- Balcony area size:
    - Remove m$^2$ (square meter)<br><br>

In [32]:
u_raw['price_in_huf']=u_raw.price_in_huf.apply(extract_num, thousand_eq='ezer', million_eq='millió', billion_eq='milliárd')
u_raw['area_size']=u_raw.area_size.apply(extract_num)
u_raw['room_lt_12_sqm']=u_raw.room.apply(lambda x: extract_num(x.split('+')[1]) if '+' in x else 0)
u_raw['room_ge_12_sqm']=u_raw.room.apply(lambda x: extract_num(x.split('+')[0]))
u_raw['room_total']=u_raw['room_ge_12_sqm']+u_raw['room_lt_12_sqm']
u_raw['balcony'] = u_raw.loc[u_raw.balcony.notnull(), 'balcony'].apply(extract_num)

- Parking lot price, column values consist of combination of currencies (HUF, EUR) and listing types (for-sale, for-rent) so it can't be analyzed properly.  
Solution: splitting into 4 categories:
    1. Sale price in HUF
    2. Monthly fee in HUF
    3. Sale price in EUR
    4. Monthly fee in EUR

In [33]:
u_raw['parking_lot_in_huf'] = np.nan
u_raw['parking_lot_in_eur'] = np.nan
u_raw['parking_lot_in_huf_monthly'] = np.nan
u_raw['parking_lot_in_eur_monthly'] = np.nan

In [34]:
not_null_flt = u_raw.parking_lot_price.notnull()
monthly_flt = u_raw.parking_lot_price.str.contains('/hó').fillna(False)
eur_flt = u_raw.parking_lot_price.str.contains('€').fillna(False)

In [35]:
u_raw.loc[not_null_flt&~eur_flt&~monthly_flt, 'parking_lot_in_huf'] = u_raw.loc[not_null_flt&~eur_flt&~monthly_flt, 'parking_lot_price'].apply(extract_num, million_eq = 'M')
u_raw.loc[not_null_flt&~eur_flt&monthly_flt, 'parking_lot_in_huf_monthly'] = u_raw.loc[not_null_flt&~eur_flt&monthly_flt, 'parking_lot_price'].apply(extract_num)
u_raw.loc[not_null_flt&eur_flt&monthly_flt, 'parking_lot_in_eur_monthly'] = u_raw.loc[not_null_flt&eur_flt&monthly_flt, 'parking_lot_price'].apply(extract_num)
u_raw.loc[not_null_flt&eur_flt&~monthly_flt, 'parking_lot_in_eur'] = u_raw.loc[not_null_flt&eur_flt&~monthly_flt, 'parking_lot_price'].apply(extract_num)

- Utilities fee:
    - Monthly cost

In [36]:
u_raw.loc[u_raw.utilities.notnull(), 'utilities'] = u_raw.loc[u_raw.utilities.notnull(), 'utilities'].apply(extract_num)
u_raw.utilities = u_raw.utilities.astype('float')

- Minimum tenancy time:
    - In years  

In [37]:
u_raw.loc[u_raw.min_tenancy =='nincs', 'min_tenancy'] = 0
months_flt = u_raw.min_tenancy.str.contains('hónap').fillna(False)
year_flt = u_raw.min_tenancy.str.contains('év').fillna(False)

In [38]:
u_raw.loc[year_flt&months_flt, 'min_tenancy'] = u_raw.loc[year_flt&months_flt, 'min_tenancy'].apply(lambda s: extract_num(s.split('év')[0]) + extract_num(s.split('év')[1])/12)
u_raw.loc[~(year_flt&months_flt)&u_raw.min_tenancy.notnull(), 'min_tenancy'] = u_raw.loc[~(year_flt&months_flt)&u_raw.min_tenancy.notnull(), 'min_tenancy'].apply(extract_num, thousand_eq = 'hónap', thousand_mlpr = 1/12)
u_raw.min_tenancy = u_raw.min_tenancy.astype('float')

Double check the data types of the interval/ratio columns:

In [39]:
u_raw[interval_or_ratio].dtypes

lat                         float64
lng                         float64
elevation                     int64
price_in_huf                float64
area_size                   float64
room                         object
balcony                     float64
parking_lot_price            object
utilities                   float64
min_tenancy                 float64
metro_lines_count           float64
trams_count                 float64
trolley_buses_count         float64
buses_count                 float64
boats_count                 float64
local_railways_count        float64
all_night_services_count    float64
dtype: object

*Room* and *parking lot price* have been splitted into sub-categories, so the original columns are redundant.

In [40]:
u_raw.drop(labels=['room', 'parking_lot_price'], axis=1, inplace=True)

---

Adding new feature to measure the relative price per square meter:

In [41]:
u_raw['price_per_sqm'] = u_raw['price_in_huf']/u_raw['area_size']

## Saving datasets with correct datatypes:

### Split listings into listing types

In [42]:
sale, rent = u_raw[sale_flt], u_raw[rent_flt]

### Split dataset into training and test
For splitting the dataset, I am using hash keys, which is calculated from unique id.  
- 20% of the data is used for model testing as a final validation step

In [43]:
train_sale, test_sale = proc.split_train_test_by_hash(sale, test_ratio=0.2, id_column='id')
train_rent, test_rent = proc.split_train_test_by_hash(rent, test_ratio=0.2, id_column='id')

In [44]:
train_sale_ratio=len(train_sale)/(len(train_sale)+len(test_sale))
print('Number of records in training set: {0:,}'.format(len(train_sale)))
print('Number of records in training set to total records {0:%}'.format(train_sale_ratio))

Number of records in training set: 34,605
Number of records in training set to total records 79.661602%


Save training and testing dataset to make it easier to use

In [45]:
fns = ['training_sale_{}.csv', 'testing_sale_{}.csv', 'training_rent_{}.csv', 'testing_rent_{}.csv']
fns_date = [n.format(SCRAPE_DATE) for n in fns]
dfs =[train_sale, test_sale, train_rent, test_rent]

In [46]:
for fn, df in zip(fns_date, dfs):
    df.to_csv(os.path.join(INT_DIR, fn), encoding='utf8', index=False)